In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [3]:
spark = SparkSession.builder.appName("lab2").config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0").getOrCreate()

24/05/10 20:04:56 WARN Utils: Your hostname, rust resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
24/05/10 20:04:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/r_usttt/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/r_usttt/.ivy2/cache
The jars for the packages stored in: /home/r_usttt/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1b1835ac-bc91-4b7f-b064-2c01f7a00405;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.13.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 300ms :: artifacts dl 12ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.13.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number

In [4]:
posts_data = (spark.read.format("com.databricks.spark.xml")
    .options(rowTag='row')
    .load('posts_sample.xml'))

programming_languages_data = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv('programming-languages.csv')

In [5]:
csv_list = programming_languages_data.selectExpr("name").rdd.flatMap(lambda x: x).collect()

In [6]:
min_year = int(posts_data.selectExpr('min(substring(_CreationDate, 0, 4))').first()[0])
max_year = int(posts_data.selectExpr('max(substring(_CreationDate, 0, 4))').first()[0])

print(min_year)
print(max_year)

2008
2019


In [7]:
def prep_lang(x, csv_list):
    language = next((language for language in csv_list if f"<{language.lower()}>" in x._Tags.lower()), None)
    return (x._Id, language) if language else None

In [8]:
result = {}

for year in range(min_year, max_year + 1):

    # filter by year
    filtered_data = posts_data.filter((col('_Tags').isNotNull()) & (col('_CreationDate').substr(0, 4) == str(year)))

    # filter None values
    mapped_data = filtered_data.rdd.map(lambda x: prep_lang(x, csv_list)).filter(lambda x: x is not None)

    aggregated_data = mapped_data \
        .map(lambda x: (x[1], 1)) \
        .reduceByKey(lambda x, y: x + y) \
        .map(lambda x: (x[0], int(x[1])))

    # convert RDD into DataFrame
    sorted_data = spark.createDataFrame(aggregated_data, ['Language', f'{year}_year']) \
        .orderBy(col(f'{year}_year').desc()) \
        .limit(10)

    result[year] = sorted_data
    result[year].show()

    sorted_data.write.parquet(f'top_pl_{year}.parquet')

+----------+---------+
|  Language|2008_year|
+----------+---------+
|      Java|        4|
|      Ruby|        3|
|JavaScript|        2|
|         C|        2|
|        Io|        1|
|    Groovy|        1|
|       PHP|        1|
|    Python|        1|
|       X++|        1|
+----------+---------+



+-----------+---------+
|   Language|2009_year|
+-----------+---------+
|       Java|       28|
|        PHP|       22|
|     Python|       22|
| JavaScript|       12|
|       Ruby|        7|
|     Delphi|        7|
|          C|        6|
|Objective-C|        5|
|    Haskell|        4|
|       Bash|        3|
+-----------+---------+



+-----------+---------+
|   Language|2010_year|
+-----------+---------+
|       Java|       52|
| JavaScript|       44|
|        PHP|       42|
|     Python|       25|
|Objective-C|       23|
|          C|       20|
|       Ruby|       11|
|     Delphi|        7|
|AppleScript|        3|
|          R|        3|
+-----------+---------+



+-----------+---------+
|   Language|2011_year|
+-----------+---------+
|        PHP|       97|
|       Java|       92|
| JavaScript|       82|
|     Python|       35|
|Objective-C|       33|
|          C|       24|
|       Ruby|       17|
|     Delphi|        8|
|       Perl|        8|
|       Bash|        7|
+-----------+---------+



+-----------+---------+
|   Language|2012_year|
+-----------+---------+
|        PHP|      136|
| JavaScript|      129|
|       Java|      124|
|     Python|       65|
|Objective-C|       45|
|          C|       27|
|       Ruby|       25|
|       Bash|        9|
|          R|        9|
|     MATLAB|        6|
+-----------+---------+



...

+-----------+---------+
|   Language|2015_year|
+-----------+---------+
| JavaScript|      270|
|       Java|      208|
|        PHP|      147|
|     Python|      119|
|          R|       43|
|          C|       38|
|Objective-C|       30|
|       Ruby|       21|
|     MATLAB|       16|
|      Scala|       13|
+-----------+---------+



+----------+---------+
|  Language|2016_year|
+----------+---------+
|JavaScript|      272|
|      Java|      179|
|    Python|      141|
|       PHP|      126|
|         R|       50|
|         C|       32|
|      Ruby|       21|
|      Bash|       16|
|     Scala|       16|
|    MATLAB|       15|
+----------+---------+



+-----------+---------+
|   Language|2017_year|
+-----------+---------+
| JavaScript|      244|
|       Java|      204|
|     Python|      185|
|        PHP|      122|
|          R|       53|
|          C|       24|
|Objective-C|       19|
|       Ruby|       16|
| PowerShell|       14|
| TypeScript|       14|
+-----------+---------+



+----------+---------+
|  Language|2018_year|
+----------+---------+
|    Python|      214|
|JavaScript|      196|
|      Java|      145|
|       PHP|       99|
|         R|       63|
|         C|       24|
|     Scala|       22|
|TypeScript|       21|
|PowerShell|       13|
|      Bash|       12|
+----------+---------+



+----------+---------+
|  Language|2019_year|
+----------+---------+
|    Python|      162|
|JavaScript|      131|
|      Java|       95|
|       PHP|       59|
|         R|       36|
|         C|       14|
|    MATLAB|        9|
|      Dart|        9|
|    Kotlin|        9|
|        Go|        9|
+----------+---------+



In [9]:
spark.stop()

In [10]:
print(pd.read_parquet('top_pl_2019.parquet'))

     Language  2019_year
0      Python        162
1  JavaScript        131
2        Java         95
3         PHP         59
4           R         36
5           C         14
6          Go          9
7      MATLAB          9
8      Kotlin          9
9        Dart          9
